## Version 0.9.6 2021 03 04

### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 The code is also hosted at https://github.com/n4rus/tbot and is licensed at 
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient
from datetime import datetime

nova_client = NovaClient(AccessKey, SecretKey)

### Importing List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

sublcq = nova_client.subs()
sublcd = sublcq['data']

blccolumns = ['Currency', 'Sub-Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)

p = 0
subids = {}
sublcs = {}

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
    
    blcdataframe.append(
        pd.Series([
            subcy,
            sublc,
        ], index = blccolumns),
        ignore_index = True)
    blcdataframe = blcdataframe.append(
    pd.Series([
        subcy,
        sublc,
    ], index=blccolumns),
        ignore_index=True)
 
    p = p + 1

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,ETH,0.2789270447
2,USDT,0.09682579
3,DGB,0.0353
4,LINK,0.0088601
5,DOGE,0.00881
6,XLM,0.00447
7,NULS,0.0044
8,IOTA,0.003905
9,XRP,0.0036


In [4]:
blcdataframe.sort_values('Sub-Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
blcdataframe

,Currency,Balance,Sub-Balance
0,ETH,NaN,0.02173153


### Symbol position, Dolar price and timestamp
For the execution to be done independently for each symbol, the below cell is needed:

In [5]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbols:
    r = 0
    ldolar = nova_client.list_trades(symbol, limit=10)
    for i in ldolar['data']:  
        fddata = nova_client.list_trades(symbol, limit=10)
        fdata = fddata['data']
        fdt = fdata[r]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        r = r + 1
        for i in fddata['data'][:1]:
            e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
            e_dataframe = e_dataframe.append(
                                                pd.Series([
                                                    symbol, 
                                                    price,
                                                    shares_to_buy
                                                    ],
                                                    index = my_columns
                                                    ),
                                                ignore_index = True)

e_dataframe

,Symbol,Currency Price,Number of Shares to Buy
0,ETH_USDT,1487.48,0.278992
1,ETH_USDT,1505.01,0.278991
2,ETH_USDT,1503.9,0.278991
3,ETH_USDT,1479.92,0.278992
4,ETH_USDT,1493.63,0.278992
5,ETH_USDT,1488.93,0.278992
6,ETH_USDT,1477.88,0.278993
7,ETH_USDT,1471.75,0.278993
8,ETH_USDT,1475.78,0.278993
9,ETH_USDT,1478.14,0.278993


### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [6]:
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Number of Shares to Buy','Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Last Orders Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Amount,Number of Shares to Buy,Last Orders Price,B/S,Momentum Price,momentum-raw-data
0,ETH_USDT,0.0188,-0.260177,1928.42,BUY,1488.93,439.49
1,ETH_USDT,0.0685,-0.210477,1953.10,BUY,1479.92,473.18
2,ETH_USDT,0.05,-0.228977,1953.10,BUY,1493.63,459.47
3,ETH_USDT,0.0186,-0.260377,1953.26,SELL,1471.75,481.51
4,ETH_USDT,0.0186,-0.260377,1953.42,SELL,1477.88,475.54
5,ETH_USDT,0.0186,-0.260376,1967.68,SELL,1475.78,491.90
6,ETH_USDT,0.0675,-0.211476,1983.00,BUY,1503.9,479.10
7,ETH_USDT,0.0674,-0.211575,1999.85,SELL,1487.48,512.37
8,ETH_USDT,0.0674,-0.211575,2034.00,SELL,1505.01,528.99


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [7]:
#orders dataframe
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listorders = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        #rework into a precise formula with momentum
        vxdata = tdata[s]
        vxi = vxdata['price']
        momentumprice = float(vxi)
        momentumraw = float(price) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        s = s + 1
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Last Orders Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp
0,ETH_USDT,0.0674,1999.85,SELL,1487.48,512.37,CANCELED,20210221 13:56:38
1,ETH_USDT,0.0674,2034.00,SELL,1505.01,528.99,CANCELED,20210220 16:08:38
2,ETH_USDT,0.0675,1983.00,BUY,1503.90,479.10,FILLED,20210220 10:51:38
3,ETH_USDT,0.0685,1953.10,BUY,1479.92,473.18,CANCELED,20210219 18:38:12
4,ETH_USDT,0.0500,1953.10,BUY,1493.63,459.47,CANCELED,20210219 18:37:37
5,ETH_USDT,0.0188,1928.42,BUY,1488.93,439.49,CANCELED,20210219 15:15:20
6,ETH_USDT,0.0186,1953.42,SELL,1477.88,475.54,FILLED,20210219 15:14:51
7,ETH_USDT,0.0186,1953.26,SELL,1471.75,481.51,CANCELED,20210219 15:14:19
8,ETH_USDT,0.0186,1967.68,SELL,1475.78,491.90,CANCELED,20210219 15:13:35


In [8]:
#define booleans
#order dataframe
z = 0
filled = 'FILLED'
statushd = {}
stdata = o_dataframe['Status']
bsdata = o_dataframe['B/S']
for i in stdata:
    bsdaty = bsdata[z]
    stdaty = stdata[z] 
    if stdaty == filled:
        statushd[f"{bsdaty}"] = f"{stdaty}"
        
    z = z + 1
    a = 9 
    b = 2
    gama = 0

In [9]:
#sort price [ok]
#wallet_amount [ok]
#dataframe momentum[ok]pt2[ok]
#buy lower(wallet_amount - amount)  [ok]   
#sell + profit(constant*market momentum)[ok]


#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}

#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders

sell = 1    
buy = 1
profit = sell - buy
# bm = buy momentum price

if momentumprice > price :
    sellprofit = math.floor(price + price/100*1/2)/1.00
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

#first block to be solved
if momentumprice < price and amount < 0.02:
    print("Market is not ready to sell")

if momentumprice < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    

Market is not ready to sell


In [10]:
#sub_account balance call
sublcq = nova_client.subs()
sublcd = sublcq['data']

p = 0
subids = {}
sublcs = {}

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
    
    p = p + 1
    print(sublcs)

{'ETH': '0.02173153'}


In [11]:
#booleans dataframe
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)

#define booleans
#order dataframe

for symbol in symbols:
    listorders = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    z = 0
    sti = 0
    filled = 'FILLED'
    statushd = {}
    stdata = o_dataframe['Status']
    bsdata = o_dataframe['B/S']
    for i in stdata:
        bsdaty = bsdata[z]
        stdaty = stdata[z] 
        if stdaty == filled:
            statushd[f"{bsdaty}"] = f"{stdaty}"
            sti = sti + 1
        z = z + 1
        a = 9 
        b = 2
        gama = 0
        for i in listorders['data'][:gama]:
            ldata = listorders['data']
            ldc = ldata[s]
            amount = ldc['amount']
            price = float(ldc['price'])
        
            amount = float(ldc['amount'])
            cprice = float(amount) * float(price)
            edolar = bcr['USDT']
            eth_wal = float(bcr['ETH']) * float(price)
            edolarwal = float(bcr['USDT']) + float(eth_wal)
            bs = ldc['side']
        
        #rework into a precise formula with momentum
            vxdata = tdata[s]
            vxi = vxdata['price']
            momentumprice = float(vxi)
            momentumraw = float(price) - float(vxi)

            tspi = float(ldc['timestamp']/1000)
            tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
            status = ldc['status']
        
 
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)
    s = s + 1
o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Last Orders Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp


### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [12]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
o_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [13]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [14]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [15]:
writer.save()